In [2]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
import matplotlib.pyplot as plt
from shapely.geometry import mapping
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
###path de yolo dentro de computadora
os.chdir(r'C:\Users\mfpen\OneDrive\Documentos\Secretaría\yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [2]:
# import warnings
# warnings.filterwarnings('ignore')
# import pandas as pd
# import math
# from shapely.geometry import Polygon
# import cv2
# from clasificacion_chinchetas import *
# import tqdm
# import numpy as np
# from osgeo import gdal
# import rasterio
# import geopandas as gpd
# import rasterio.mask
# import sys
# from shapely.geometry import mapping
# from shapely import geometry
# import cv2
# import numpy as np

## Raster y shape deteccion

In [20]:
raster=r"C:\Users\mfpen\Pictures\Ixtapan_Sal.tif"
src=rasterio.open(raster)

deteccion=r"D:\Secretaría\salida_detecciones\Ixtapan\Ixtapan_Sal_casas.shp"
deteccion_shp=gpd.read_file(deteccion)


In [21]:
deteccion_shp

,clase_dete,conf,area,geometry
0,establecimiento,0.48738,208.166051,"POLYGON ((-11105765.130 2135576.959, -11105744..."
1,casas,0.63394,422.365900,"POLYGON ((-11105771.332 2135596.202, -11105753..."
2,establecimiento,0.27702,56.315453,"POLYGON ((-11105762.218 2135574.926, -11105754..."
3,establecimiento,0.32621,160.901295,"POLYGON ((-11105759.917 2135568.273, -11105750..."
4,establecimiento,0.47255,217.216749,"POLYGON ((-11105752.240 2135574.485, -11105740..."
...,...,...,...,...
29203,casas,0.74758,120.675972,"POLYGON ((-11088680.303 2135203.243, -11088669..."
29204,establecimiento,0.27273,290.627965,"POLYGON ((-11088594.885 2136545.562, -11088578..."
29205,establecimiento,0.55595,231.295612,"POLYGON ((-11088638.886 2136507.708, -11088629..."
29206,establecimiento,0.28596,464.602490,"POLYGON ((-11088587.043 2136570.300, -11088574..."


In [22]:
def ampliar_shape(shape,factor_ampliacion=2):
    """Amplifica el polygon de cada manzana con el fin de extrar imagenes sin perder informacion de la manzana"""
    shape["geometry"]=shape["geometry"]
    shape['centroid']=shape.centroid
    geometry=[]
    clase=[]
    for i,polygon in enumerate(shape['geometry']):
        # try:
        point=mapping(shape['centroid'][i]).get('coordinates')
        # except:
        #     continue
        x=point[0]
        y=point[1]
        go=[]
        coodinates=mapping(polygon).get('coordinates')[0]
        for a in coodinates:
            x1=a[0]
            y1=a[1]
            x2=x+(x1-x)*factor_ampliacion
            y2=y+(y1-y)*factor_ampliacion
            go.append((x2,y2))
        geometry.append(Polygon(go))
        clase.append(shape.loc[i,"clase_dete"])
    return gpd.GeoDataFrame({"clase_dete":clase},geometry=geometry)

In [23]:
deteccion_shp=deteccion_shp.dropna(how="all").reset_index(drop=True)
angulo_manzana=[]
for manzana in range(len(deteccion_shp)):
    proyecciones1=mapping(deteccion_shp['geometry'][manzana]).get('coordinates')
    angulos=[]
    d=[]
    poly=pd.DataFrame(proyecciones1[0])
    for point in range(1,len(poly)):
        d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
#         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
#     angulo_manzana.append(angulos[d.index(max(d))])
# deteccion_shp['angulo']=angulo_manzana
deteccion_shp1=ampliar_shape(deteccion_shp,factor_ampliacion=1.25)
len(deteccion_shp1)

## Clasificar casas y/o terrenos (Alexnet)

##### Pesos y diccionario Alexnet

In [7]:
# weights=r'C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint/best_mod11.pth'
# num_classes=5
# diciconario={0: 'casas', 1: 'en_construccion', 2: 'establecimiento', 3: 'multivivienda', 4: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)


# weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell11.pth"
# num_classes=6
# diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

#### Clasificación

In [7]:
def normalizar_array(img):
    return (img-np.min(img))/(np.max(img)-np.min(img))

In [8]:
deteccion_shp.dropna(how="all")

,clase_dete,conf,area,index_righ,Clase,geometry,centroid
0,establecimiento,0.38509,88.495712,NaN,NaN,"POLYGON ((-11096040.209 2137119.002, -11096032...",POINT (-11096037.652 2137112.677)
1,establecimiento,0.24774,377.112411,NaN,NaN,"POLYGON ((-11096062.852 2137077.681, -11096038...",POINT (-11096048.802 2137073.659)
2,casas,0.40355,343.926519,NaN,NaN,"POLYGON ((-11096057.714 2137046.864, -11096039...",POINT (-11096046.180 2137040.610)
3,establecimiento,0.58628,165.929461,NaN,NaN,"POLYGON ((-11096047.771 2137048.491, -11096037...",POINT (-11096040.499 2137042.652)
4,establecimiento,0.35066,88.495712,NaN,NaN,"POLYGON ((-11096042.793 2137011.312, -11096035...",POINT (-11096037.346 2137007.209)
...,...,...,...,...,...,...,...
2019,establecimiento,0.53625,90.506979,NaN,NaN,"POLYGON ((-11094322.136 2136684.159, -11094312...",POINT (-11094316.238 2136680.889)
2020,establecimiento,0.56976,80.450648,NaN,NaN,"POLYGON ((-11094324.692 2136690.775, -11094314...",POINT (-11094318.906 2136687.994)
2021,establecimiento,0.79482,90.506979,NaN,NaN,"POLYGON ((-11094327.661 2136708.106, -11094317...",POINT (-11094321.763 2136704.836)
2022,establecimiento,0.84896,90.506979,NaN,NaN,"POLYGON ((-11094331.752 2136655.341, -11094323...",POINT (-11094325.367 2136653.182)


In [9]:
deteccion_shp1

,clase_dete,geometry
0,establecimiento,"POLYGON ((-11096040.848 2137120.583, -11096031..."
1,establecimiento,"POLYGON ((-11096066.365 2137078.686, -11096036..."
2,casas,"POLYGON ((-11096060.597 2137048.427, -11096037..."
3,establecimiento,"POLYGON ((-11096049.589 2137049.951, -11096036..."
4,establecimiento,"POLYGON ((-11096044.154 2137012.337, -11096034..."
...,...,...
2019,establecimiento,"POLYGON ((-11094323.611 2136684.976, -11094311..."
2020,establecimiento,"POLYGON ((-11094326.138 2136691.471, -11094313..."
2021,establecimiento,"POLYGON ((-11094329.136 2136708.923, -11094316..."
2022,establecimiento,"POLYGON ((-11094333.348 2136655.881, -11094323..."


In [10]:
%matplotlib
import tqdm
import cv2
geometry=[]
status=[]
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    imagen=cv2.cvtColor(np.moveaxis(array,0,-1),cv2.COLOR_RGB2BGR)
    # cv2_imshow(imagen)
    imagen=np.array(imagen,dtype='float32')/255
    img=imagen.copy()
    # img=np.array(img,"float")
    # for channel in range(img.shape[2]):
    for y in range(1,img.shape[0]-1):
        for x in range(1,img.shape[1]-1):
            for w in range(-1,2):
                for h in range(-1,2):
                    if np.min(np.abs(img[y+h,x+w,:]-img[y,x,:]))*255<=20:
                        # print(y,x,w,h)
                        img[y+h,x+w,:]=img[y,x,:]
                       # print(imagen[y,x,:])
    # print(img.shape)
    
    # maximo=np.max(img)
    # minimo=np.min(img)
    # cv2_imshow(img)
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray=(normalizar_array(gray)*2-1)*255
    # plt.imshow(gray)
    # break
    y=gray.copy()
    y[y<5]=0
    y[y>=5]=255
    gray=cv2.erode(y,(6,6))
    # plt.imshow(gray)
    # break
    # cv2_imshow("y",np.array(np.hstack([gray,imagen[:,:,0]]),dtype="uint8"))
    # img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # img=(np.array(img[:,:,1],dtype=float)*255)**2
    img=(np.array(y,dtype=float)*255)**2
    # break
    h,w= img.shape
    # plt.imshow(gray)
    # break
    yc,xc=int(h/2),int(w/2)
    thres=img.copy()
    u=190
    thres[thres<u**2]=0
    thres[thres>=u**2]=255
    thres=np.array(thres,dtype="uint8")#_,thres=cv2.threshold(img,173,255,1)
    # plt.imshow(thres)
    # break
    f=1.25
    kernel = np.array([[-1,-1,1],[-1,1,-1],[1,-1,-1]])
    kernel=np.array([[1,1,1],[0,0,1],[0,0,1]])
    dst = cv2.filter2D(thres,-1,kernel)
    cont,hist=cv2.findContours(dst,2,1)
    res=pd.DataFrame({"c":cont,"area":[cv2.contourArea(cnt) for cnt in cont]})
    res.sort_values(by="area",ascending=False,inplace=True)
    res=res.reset_index(drop=True)
    try:
        cnt=res.c.values[0]
    except:
        geometry.append(polygo)
        status.append("error")
        print("aca")
        continue
    if res.loc[0,"area"]<.5*polygo.area:
        geometry.append(polygo)
        status.append("area_small")
        print("aqui")
        continue
    epsilon = 0.03*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    cnt=approx
    s=np.zeros_like(dst)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    coords=shapes[0].get("coordinates")[0]
    left,bottom=np.min(coords,axis=0)
    right,top=np.max(coords,axis=0)
    height=imagen.shape[0]
    width=imagen.shape[1]
    cv2.drawContours(s,[box],0,(127),2)
    cv2.drawContours(s,[cnt],-1,(255),-1)

    geometry.append(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))
    status.append("correcto")
    # cv2_imshow(s)
    # plt.show()
    #break
    # gray = np.float32(gray)
    # dst = cv.cornerHarris(gray,2,3,0.04)
    # dst[dst<=0]=0
    # dst=dst
    # sa=dst.copy()
    # sa=sa.reshape(-1)
    # sa.sort()
    # box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
    # # print(box)
    # box=np.stack([box[:,1],box[:,0]],axis=-1)
    # # box
    # # cv2.drawContours(img,[box],0,(0,0,255),2)
    # cv2_imshow("s",img)
#     rows,cols = img.shape[:2]
#     [vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
#     lefty = int((-x*vy/vx) + y)
#     righty = int(((cols-x)*vy/vx)+y)
#     cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
#     cv2_imshow("s",img)
#     # four_images=[array[2],array[1],array[0]]
#     # stacked_images = np.stack(four_images, axis=-1)
    
# #     imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)
# #     clase,imagen=model_class.predict_image(imagen_1)
    
# #     cv2.|imshow("im",imagen)
# #     cv2.waitKey()
# #     cv2.destroyAllWindows()
#     deteccion_shp.loc[i,"clase_n"]=clase

Using matplotlib backend: <object object at 0x000001D3E8737D80>


  1%|          | 21/2024 [00:14<33:17,  1.00it/s]

aqui


  8%|▊         | 172/2024 [01:35<15:51,  1.95it/s]

aqui


...

 94%|█████████▍| 1904/2024 [21:15<01:14,  1.62it/s]

aqui


 94%|█████████▍| 1905/2024 [21:18<02:44,  1.38s/it]

aqui


100%|██████████| 2024/2024 [24:30<00:00,  1.38it/s]


In [11]:
deteccion_shp['status']= status
deteccion_shp['geometry']=geometry

In [18]:
deteccion_shp.drop(['centroid'],axis=1,inplace=True)

In [19]:
deteccion_shp.to_file(r"D:\Secretaría\prueba_rotacion/prueba_deteccion_1_rotacion.shp")

In [11]:
img

array([[          0,           0,           0, ...,           0,           0,           0],
       [          0,           0,           0, ...,           0,           0,           0],
       [          0,           0,           0, ...,           0,           0,           0],
       ...,
       [          0,           0,           0, ...,           0,           0,           0],
       [          0,           0,           0, ...,           0,           0,           0],
       [          0,           0,           0, ...,           0,           0,           0]])

In [12]:
gpd.GeoSeries(geometry).plot()

<AxesSubplot:>

In [13]:
ñ

NameError: name 'ñ' is not defined

In [ ]:
type(array)

In [ ]:
(np.array([1.1,2.1,3.1,4.1],dtype='float32'))

In [ ]:
gpd.GeoDataFrame({"status":status},geometry=geometry,crs=3857).to_file(r"E:\rastermunicpial/test_FER_nuevarotacion.shp")


In [ ]:
res.loc[i,"c"]

#  box_l.append(Polygon([(map_d(x[0],0,width,width_min,width_max),map_d(x[1],heigth,0,heigth_min,height_max)) for x in box]))
coords=shapes[0].get("coordinates")[0]
left,bottom=np.min(coords,axis=0)
right,top=np.max(coords,axis=0)


height=imagen.shape[0]
width=imagen.shape[1]
left,bottom,right,top,height,width


           

In [ ]:
geomtry=(Polygon([(map_d(x[0],0,width,left,right),map_d(x[1],height,0,bottom,top)) for x in box]))

In [ ]:
deteccion_shp

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
# img=cv2.imread("Alarms/malavi/industria~2.png")
# img=cv2.resize(img,(60,60))
# img=cv2.resize(img,(100,100))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=np.array(img[:,:,1],dtype=float)**2
h,w= img.shape
yc,xc=int(h/2),int(w/2)
thres=img.copy()
u=190
thres[thres<u**2]=0
thres[thres>=u**2]=255
thres=np.array(thres,dtype="uint8")#_,thres=cv2.threshold(img,173,255,1)
f=1.25
kernel = np.array([[-1,-1,1],[-1,1,-1],[1,-1,-1]])
kernel=np.array([[1,1,1],[0,0,1],[0,0,1]])
dst = cv2.filter2D(thres,-1,kernel)
cont,hist=cv2.findContours(dst,2,1)
res=pd.DataFrame({"c":cont,"s":[cv2.contourArea(cnt) for cnt in cont]})
res.sort_values(by="s",ascending=False,inplace=True)
cnt=res.c.values[0]
epsilon = 0.02*cv2.arcLength(cnt,True)
approx = cv2.approxPolyDP(cnt,epsilon,True)
cnt=approx
s=np.zeros_like(dst)
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(s,[box],0,(127),2)
cv2.drawContours(s,[cnt],-1,(255),-1)
plt.imshow(s)
plt.show()
print(cont)

In [ ]:
y=gray.copy()
y[y<200]=0
y=cv2.erode(y,(6,6))
cv2_imshow("y",y)

In [ ]:
pd.DataFrame(gray).to_clipboard()

In [ ]:
import numpy as np
import cv2 as cv
filename = 'chessboard.png'
img =imagen.copy()
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)
dst[dst<=0]=0
dst=dst**2
sa=dst.copy()
sa=sa.reshape(-1)
sa.sort()
box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
print(box)
box=np.stack([box[:,1],box[:,0]],axis=-1)
# box
# cv2.drawContours(img,[box],0,(0,0,255),2)
# cv2_imshow("s",img)
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
cv2_imshow("s",img)
#result is dilated for marking the corners, not important
# dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
# img[dst>0.01*dst.max()]=
img[dst>0.05*dst.max()]=[0,0,255]
cv2_imshow('dst',img)
# if cv.waitKey(0) & 0xff == 27:
#     cv.destroyAllWindows()

In [ ]:
%matplotlib

xp, yp,_= img.shape

x = np.arange(0, xp, 1)
y = np.arange(0, yp, 1)
Y, X = np.meshgrid(y, x)
fig=plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X,Y,dst,c=np.random.randint(0,255,len(dst.reshape(-1)))/255)

In [ ]:
sa=dst.copy()
sa=sa.reshape(-1)
sa.sort()
box=np.moveaxis(np.where(dst>=sa[-4]),0,-1)
print(box)
box=np.stack([box[:,1],box[:,0]],axis=-1)
# box
# cv2.drawContours(img,[box],0,(0,0,255),2)
# cv2_imshow("s",img)
rows,cols = img.shape[:2]
[vx,vy,x,y] = cv.fitLine(box, cv.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)
cv2_imshow("s",img)

## Clasificar chinchetas (Matching template)

#### Plantillas

In [ ]:
# No borrar path compañeros

path_ruben = r'C:\Users\ruben\Desktop\GEM\Geo\Bases\Plantillas'
path_hector = '/home/hector/Documentos/Infis/Geo/Data/Plantillas'
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))

#### Clasificación

In [ ]:
import tqdm
import cv2
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    four_images=[array[2],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    
    imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)

    # clasificacion
    clase_aux, n_aux, umbral_aux  = iter_umbral_fn(imagen_1, dict_gris,n=100,salto_n=10,
                                                    umbral=0.94,min_umbral=0.8)
    
    # Revisamos la mayor confusion que se da ... areas_verdes ~ establecimiento_google
    if clase_aux in ['a_verdes','establecimiento_google']:
        clase_aux = areasv_vs_estabgoogle(imagen_1, dict_gris)    
  
    deteccion_shp.loc[i,"clase_chincheta"] = clase_aux
    deteccion_shp.loc[i,"umbral"] = umbral_aux
    deteccion_shp.loc[i,"n_resize"] = n_aux